In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.io as sio
from PIL import Image
import admm

In [ ]:
# DATA_PATH = '../../../OneDrive - m.titech.ac.jp/Lab/data'
DATA_PATH = '../data'
n = 128
m = 256
tau = 1e-3

In [ ]:
def createDrgb(n):
    I = sp.eye(n**2, format="lil")

    Dx = I - sp.lil_matrix(np.roll(I.toarray(), 1, axis=1))
    Dx[n - 1 :: n, :] = 0
    Dy = I - sp.lil_matrix(np.roll(I.toarray(), n, axis=1))
    Dy[-n:, :] = 0
    D0 = sp.lil_matrix((n**2, n**2))

    D = sp.block_array([[Dx, D0, D0], [D0, Dx, D0], [D0, D0, Dx], [Dy, D0, D0], [D0, Dy, D0], [D0, D0, Dy]])

    return D


def createDmono(n):
    I = sp.eye(n**2, format="lil")

    Dx = I - sp.lil_matrix(np.roll(I.toarray(), 1, axis=1))
    Dx[n - 1 :: n, :] = 0
    Dy = I - sp.lil_matrix(np.roll(I.toarray(), n, axis=1))
    Dy[-n:, :] = 0

    D = sp.block_array([Dx, Dy])

    return D


# D = createDrgb(n
D = createDmono(n)

In [ ]:
captured = Image.open(f"{DATA_PATH}/capture_230516/Cameraman_edited.png")
# captured = captured.crop((400, 460, 860, 920)).resize((n, n))
captured = np.array(captured)
g = captured.reshape(-1, 1)

In [ ]:
SETTING = "hadamard_FISTA_l122_p-10_lmd-100"
# H = np.load(f"{DATA_PATH}/systemMatrix/H_matrix_{SETTING}.npy")
loaded_data = sio.loadmat(f"{DATA_PATH}/240818/systemMatrix/H_matrix_{SETTING}.mat")
H = loaded_data['matrix']

In [ ]:
admm = admm.Admm(H, g, D, tau)
f, obj, err = admm.solve()